#Sistema di raccomandazione
Il sistema implementa un approccio di raccomandazione one-time, evitando la costruzione di matrici di similarità NxN. Applica un filtro hard sulla categoria e combina una similarità basata su ingredienti con una funzione deterministica di brand e segmento, pesate rispettivamente 0.8 e 0.2.



In [1]:
import pandas as pd
import numpy as np



1) Importo il df e creo un nuovo df con le colonne d'interesse



In [6]:
df = pd.read_csv("brand_categoria.csv",dtype={"code": str})

df = df[['code', 'product_name','brand_name', 'brand_segment','macro_category']].copy()
df['code'] = df['code'].astype(str)
df['product_name'] = df['product_name'].astype(str)
df['brand_name'] = df['brand_name'].astype(str)
df['brand_segment'] = df['brand_segment'].astype(str)
df['macro_category'] = df['macro_category'].astype(str)


df.head(20)



,code,product_name,brand_name,brand_segment,macro_category
0,5013965698897,1,1,mass_market,Other
1,88582000939,2,1,mass_market,Other
2,0650240025396,zan zusi b b flash 30,1,mass_market,Other
3,3560070084074,deo men marine,1 de carrefour,mass_market,Deodorants
4,6134598000044,savon liquide main life,100 da,mass_market,Hygiene
5,5943044010046,gerocossen,12,mass_market,Other
6,20546472,cien,12,mass_market,Other
7,3700216252688,glycerine vegetale,123gelules,mass_market,Other
8,8859423200212,habino,13000,mass_market,Other
9,6130460000747,nan,150da,mass_market,Other


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
df_ing = pd.read_csv("ingr_cat.csv",dtype={"code": str})


df_ing['code'] = df_ing['code'].astype(str)
df_ing['product_name'] = df_ing['product_name'].astype(str)
df_ing['macro_category'] = df_ing['macro_category'].astype(str)
df_ing['inci_name'] = df_ing['inci_name'].astype(str)

df_finale = (
    df_ing
    .groupby(['code', 'product_name', 'macro_category'])['inci_name']
    .apply(lambda lst: ", ".join(sorted(set(lst))))
    .reset_index()
    .rename(columns={'inci_name': 'ingredients'}))

df_finale.head(20)


,code,product_name,macro_category,ingredients
0,00000014,Süßlupinen Mehl,Other,"Arnica Montana, Avoid Contact With Eyes, Burit..."
1,00000311,Huile végétale Amande douce,Oils,Oil
2,00000511,pure collagène marin,Other,"Added Sugarss, Colorings, Cr 6Ad, Croydon, Lac..."
3,00000561,Huile végétale coco,Oils,Oil
4,00000587,Macérât huileux Lys,Oils,"Lilium Candidum Flower Extract, Oil"
5,00000695,olé olé Aloé,Other,"Allantoin, Aloe Barbadensis Leaf Juice, Benzoi..."
6,00000859,Base lavante,Makeup,"Coco-Glucoside, Decyl Glucoside, Glycerin, Gly..."
7,00000905,Henné d'Egypte,Other,Lawsonia Inermis Leaf Powder
8,00001277,Indian Healing Clay,Other,Natural Calcium Bentonite Clay
9,0000160332625,Colgate toothpaste,Hygiene,"Cellulose Gum, Citric Acid, Cocamidopropyl Bet..."


**2. Assegnazione pesi ai brand_segment.**

Il sistema di suggerimento tiene conto del segmento di mercato e pesa la similarità in base ad esso.
Prodotti dello stesso segmento avranno peso maggiore.

In [8]:
SEGMENT_SIMILARITY = {
    ('middle', 'middle'): 0.7,
    ('mass_market', 'mass_market'): 0.7,
    ('luxury', 'luxury'): 0.7,

    ('mass_market', 'middle'): 0.5,
    ('middle', 'mass_market'): 0.5,

    ('mass_market', 'luxury'): 0.3,
    ('luxury', 'mass_market'): 0.3,

    ('middle', 'luxury'): 0.5,
    ('luxury', 'middle'): 0.5
}


**3. Creazione dell'array di similarità 1xK.**

Per il brand non uso una cosine similarity, ma una funzione deterministica basata su regole di business che tengono conto sia dell’identità del brand sia del segmento di mercato.

Il sistema applica un hard filter sulla categoria per ridurre lo spazio di ricerca e garantire coerenza semantica, e successivamente calcola una similarità pesata basata su ingredienti e su regole di brand e segmento.



In [9]:
def brand_segment_similarity_1xK(
    query_product_id,
    df,
    category_col='macro_category',
    segment_col='brand_segment'
):
    query_product_id = str(query_product_id)

    if query_product_id not in df['code'].values:
        raise ValueError("Codice prodotto non trovato")

    query_row = df[df['code'] == query_product_id].iloc[0]
    query_category = query_row[category_col]
    query_brand = query_row['brand_name']
    query_segment = query_row[segment_col]

    # Filtro hard: stessa categoria -> ricerca tra prodotti della stessa categoria
    df_filtered = df[df[category_col] == query_category].copy()
    df_filtered = df_filtered[df_filtered['code'] != query_id]
    #filtro per evitare che restituisca lo stesso prodotto

    similarities = []

    for _, row in df_filtered.iterrows():
        if row['brand_name'] == query_brand:
            similarities.append(1.0)#se il prodotto appartiene allo stesso brand avrà similarità massima
        else:
            pair = (query_segment, row[segment_col]) #altrimenti considera il segmento di mercato
            similarities.append(SEGMENT_SIMILARITY.get(pair, 0.0))

    return np.array(similarities), df_filtered




4. Applicazione della funzione e creazione dell'aray one-time a partire dal prodotto


In [68]:
query_id = input('Inserisci il codice del prodotto: ')

brand_sim = brand_segment_similarity_1xK(query_id, df)
#non ottengo una matrice di similarità NXN, ma un array 1xK:
#cioè la similarità del prodotto query rispetto a tutti gli altri prodotti della stessa categoria



#brand_sim è una tupla: il primo elemento è l'array 1xK, il secondo elemento è il dataframe con i prodotti candidati
#brand_sim == (similarity_array, df_filtered).
#la tupla serve a sapere a quali prodotti mi riferisco
brand_sim_array, df_brand_candidates = brand_sim



Inserisci il codice del prodotto: 8029513115935


In [69]:
brand_sim_array[:10] #vedo i primi dieci valori di similarità

array([0.7, 0.5, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7])

In [70]:
df_brand_candidates.head(10) #vedo i primi dieci candidati


,code,product_name,brand_name,brand_segment,macro_category
4,6134598000044,savon liquide main life,100 da,mass_market,Hygiene
24,99482463069,foaming hand soap,365 whole foods,middle,Hygiene
25,99482496517,mint garden hand soap,365 whole foods market,mass_market,Hygiene
28,4987176082152,oral b,3d white,mass_market,Hygiene
37,4102968000415,dentifrice,50 ml,mass_market,Hygiene
58,3770018286228,dentifrice a croquer menthe douce,900 care,mass_market,Hygiene
60,3770018286013,dentifrice arome menthe extra fraiche,900 care,mass_market,Hygiene
61,3760346073027,dentifrice en pastilles fraise,900 care,mass_market,Hygiene
62,3760346074611,dentifrice en pastille,900 care,mass_market,Hygiene
65,3770018286235,dentifrice a croquer menthe extra fraiche,900 care,mass_market,Hygiene


5. Con questa funzione si trovano i prodotti più simili a un prodotto dato, guardando solo gli ingredienti. Prima limita il confronto ai prodotti della stessa categoria, così il confronto ha senso.
Poi confronta gli ingredienti del prodotto scelto con quelli degli altri e dice quanto si assomigliano, usando un punteggio numerico.

In [71]:
def ingredient_similarity_1xK_detailed(
    query_product_id,
    df_finale,
    X_ing,      # matrice bag-of-words
    category_col='macro_category'
):
    query_product_id = str(query_product_id)

    if query_product_id not in df_finale['code'].values:
        raise ValueError("Codice prodotto non trovato")

    query_row = df_finale[df_finale['code'] == query_product_id].iloc[0]
    query_category = query_row[category_col]

    # Filtro hard: stessa categoria
    df_filtered = df_finale[df_finale[category_col] == query_category].copy()
    df_filtered = df_filtered[df_filtered['code'] != query_product_id]

    print(f"Trovati {len(df_filtered)} prodotti nella stessa categoria")

    idx_query = df_finale.index[df_finale['code'] == query_product_id][0]

    # Vettore query
    vec_query = X_ing[idx_query]

    # Vettori filtrati
    indices_filtered = df_filtered.index
    X_filtered = X_ing[indices_filtered]

    # Similarità coseno 1xK
    similarities = cosine_similarity(vec_query, X_filtered)[0]

    return np.array(similarities), df_filtered



In [72]:

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

vectorizer = CountVectorizer(token_pattern=r'[^,]+')
X_ing = vectorizer.fit_transform(df_finale['ingredients'])



In [73]:
query_id = input('Inserisci il codice del prodotto: ')
ing_sim = ingredient_similarity_1xK_detailed(query_id, df_finale, X_ing)
ing_sim_array, df_ing_candidates = ing_sim



Inserisci il codice del prodotto: 8029513115935
Trovati 1457 prodotti nella stessa categoria


In [74]:
ing_sim_array[:10]

array([0.        , 0.11396058, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.25197632, 0.10482848])

In [75]:
df_ing_candidates.head(10)

,code,product_name,macro_category,ingredients
9,0000160332625,Colgate toothpaste,Hygiene,"Cellulose Gum, Citric Acid, Cocamidopropyl Bet..."
22,0000902980642,Curasept colluttorio,Hygiene,"Aroma, Ascorbic Cid, Chlorhexidine Digluconate..."
26,0001182258268,Oral care dry mouth lozenges fruit mix,Hygiene,"Blue 1, Blue 2, Gum Arabic, Lime, Oil, Orange,..."
34,00014478,Luminous white,Hygiene,Benzyl Alcoholi Alcohol Benc0Lico
36,00014647,Colgate sensitiva pro alivio,Hygiene,"Aromasabor, Interrompa A5 Ote Nal, Kanthan Gum..."
37,00014806,Colgate Pasta Dental,Hygiene,"Aroma, Benzyl Alcohol, Cellulose Gum, Ci 77891..."
52,00037541,Ultimate Mouth Wash,Hygiene,"900 500, Anethole, Citrus Paradisi Seed Extrac..."
74,0009253900004,burts beevfor kids fluoride toothpaste,Hygiene,"Carrageenan, Cut, Hydrated Silica, Or Missing ..."
94,0010486007004,The Grandpa Soap,Hygiene,"Glycerin, Pinus Palustris Wood Tar, Sodium Chl..."
124,0011111377462,Soap Bar,Hygiene,"Cocamidopropyl Betaine, Dipropylene Glycol, La..."


**5. Indicizzazione su codice prodotto**

Costruzione nuovo DF: unione dei due dataframe sul codice prodotto al fine del calcolo della similarità finale.

In [76]:
df_ing = df_ing_candidates[['code']].copy()
df_ing['ing_sim'] = ing_sim_array

df_brand = df_brand_candidates[['code']].copy()
df_brand['brand_sim'] = brand_sim_array

#allineo la similarità
df_merged = df_ing.merge(
    df_brand,
    on='code',
    how='inner'
)


In [77]:
#check per verificare che l'allineamento è corretto
assert len(df_merged) > 0
assert df_merged['code'].is_unique


**6. Calcolo della similarità finale**

Gli ingredienti hanno peso maggiore nel determinare il prodotto più simile (0.8). I brand hanno peso minore (0.2).


In [78]:
df_merged['final_similarity'] = (
    0.8 * df_merged['ing_sim'] +
    0.2 * df_merged['brand_sim']
)


**6. Costruzione dataframe per la visualizzazione del prodotto più simile**


In [ ]:
query_product_name = (
    df.loc[df['code'] == query_id, 'product_name']
    .iloc[0]
)
risultato = pd.DataFrame({
    'query_product_id': query_id,
    'query_product_name': query_product_name,
    'recommended_product_id': df_merged['code'],
    'final_similarity_score': df_merged['final_similarity'],

})

#ordino i prodotti per suggerire il prodotto più simile
risultato = risultato.sort_values(
    by='final_similarity_score',
    ascending=False
).reset_index(drop=True)

risultato['rank'] = risultato.index + 1

# arricchimento informativo (nome, brand, segmento)
risultato = risultato.merge(
    df[['code', 'product_name', 'brand_name', 'brand_segment']],
    left_on='recommended_product_id',
    right_on='code',
    how='left'
).drop(columns='code')

risultato = risultato[
    [
        'rank',
        'query_product_id',
        'query_product_name',
        'recommended_product_id',
        'product_name',
        'brand_name',
        'brand_segment',
        'final_similarity_score'
    ]
]

risultato.head(10)



,rank,query_product_id,recommended_product_id,product_name,brand_name,brand_segment,final_similarity_score
0,1,8029513115935,0628451857143,savon a mains,the unscented company,mass_market,0.595842
1,2,8029513115935,4056489294566,gentle pure hand soap,cien,mass_market,0.543162
2,3,8029513115935,5701017418615,hand soap,sensitive for you,mass_market,0.522473
3,4,8029513115935,20827397,sensitive hand soap,cien,mass_market,0.522473
4,5,8029513115935,7072463113170,refill hand wash white tea verbena 500ml 1 stk,elle basic as,mass_market,0.489149
5,6,8029513115935,7090025533099,moisturising hand wash white tea verbena 500 ml,elle basic as,mass_market,0.489149
6,7,8029513115935,7311043009695,hand soap 0 perfume colorants,mevolution sensitive,mass_market,0.482857
7,8,8029513115935,0732913229307,free clean soap,seventh generation,mass_market,0.475451
8,9,8029513115935,3256228666741,savon vegetal ressourcant peche jasmin bio,u bio,mass_market,0.463249
9,10,8029513115935,0829576020058,lemon mint liquid hand soap,ever spring,mass_market,0.463249
